In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from MyEarlyStopping import MyEarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  
import pandas as pd
import timeit

In [3]:
All_data = ImageDataGenerator(preprocessing_function=preprocess_input)
All_data_generator = All_data.flow_from_directory('/data/home/cou/yuli5/Audio/online_data_adaption/Spec_Audio_folder_digital',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 30000 images belonging to 10 classes.


In [4]:
image_names = All_data_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = All_data_generator.classes
All_participant_class = tf.keras.utils.to_categorical(ALL_participant_class-1, num_classes=60)
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
All_command_uniform = All_command_class*0+1/10

In [5]:
All_Inputs = [next(All_data_generator)[0][0] for _ in range(len(All_data_generator))]
All_Inputs = np.array(All_Inputs)

In [6]:
# image_no<=20 means every word repeat 20 times 
s_number = 5 # ALL_participant_class  the subject index

select_indexs_train = (image_no<=19)&(ALL_participant_class<=s_number)
Train_Inputs = All_Inputs[select_indexs_train]
Train_participant_class = All_participant_class[select_indexs_train][:,0:s_number]
Train_participant_uniform = Train_participant_class*0+1/s_number
Train_command_class = All_command_class[select_indexs_train]
Train_command_uniform = Train_command_class*0+1/10

In [7]:
select_indexs_val = (image_no>29)&(image_no<=39)&(ALL_participant_class<=s_number)
Val_Inputs = All_Inputs[select_indexs_val]
Val_participant_class = All_participant_class[select_indexs_val][:,0:s_number]
Val_participant_uniform = Val_participant_class*0+1/s_number
Val_command_class = All_command_class[select_indexs_val]
Val_command_uniform = Val_command_class*0+1/10

In [8]:
select_indexs_test = (image_no>39)&(image_no<=49)&(ALL_participant_class<=s_number)
Test_Inputs = All_Inputs[select_indexs_test]
Test_participant_class = All_participant_class[select_indexs_test][:,0:s_number]
Test_participant_uniform = Test_participant_class*0+1/s_number
Test_command_class = All_command_class[select_indexs_test]
Test_command_uniform = Test_command_class*0+1/10

In [9]:
Perfomance = pd.DataFrame()

In [10]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


In [11]:
high_acc = 0
for run in range(0,10):
    
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer

    ###################### model 1 ###################### 
    layer_1_0 = tf.keras.layers.Dense(d,name="weight_1")(resnet_model.output) #times weight before flatten
    layer_1_1 = tf.keras.layers.Flatten(name='flatten_1')(layer_1_0)

    Dense_1_1 = tf.keras.layers.Dense(shape_1_1, activation=actv_fun_1_1,name='fc1_1')
    layer_1_2 = Dense_1_1(layer_1_1)
    Dense_1_2 = tf.keras.layers.Dense(shape_1_2, activation=actv_fun_1_2,name='fc1_2')
    layer_1_3 = Dense_1_2(layer_1_2)

    Dense_1_3 = tf.keras.layers.Dense(s_number, activation='softmax' ,name='participant_output')
    out_layer_1 = Dense_1_3(layer_1_3)

    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)

    ###################### model 1' ###################### 
    layer_1_2_  = Dense_2_1(layer_1_1)
    layer_1_3_  = Dense_2_2(layer_1_2_)
    out_layer_1_ = Dense_2_3(layer_1_3_)

    ###################### model 1' ###################### 
    layer_2_2_  = Dense_1_1(layer_2_1)
    layer_2_3_  = Dense_1_2(layer_2_2_)
    out_layer_2_ = Dense_1_3(layer_2_3_)

    resnet_model = tf.keras.Model(resnet_model.input, [out_layer_1,out_layer_2,out_layer_1_,out_layer_2_])

    # resnet_model.summary() 

    w_1, w_2, w_1_, w_2_ = 1,1,1,1
    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [MyEarlyStopping(monitor1 = 'val_' + resnet_model.layers[-1].name+'_accuracy',
                                  monitor2 = 'val_' + resnet_model.layers[-2].name+'_accuracy',
                                  patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss=["categorical_crossentropy","categorical_crossentropy","mse","mse"],
                         loss_weights=[w_1, w_2, w_1_, w_2_], metrics=['accuracy'])
    start = timeit.default_timer()
    history1 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)

    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
    history2 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)
   
    stop = timeit.default_timer()
    run_time = stop - start
    
    predictions = resnet_model.predict(Test_Inputs)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    participant_class = [i.split("/")[1].split("_")[1] for i in All_data_generator.filenames]
    participant_class = np.array(participant_class)[select_indexs_test]
    participant_class = np.array(list(map(int, participant_class)))
    acc_s = round(sum(x == y for x, y in zip(participant_class, predicted_classes)) / len(participant_class),4)
    
    predictions = resnet_model.predict(Test_Inputs)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.array([i.split("/")[0] for i in All_data_generator.filenames])
    true_classes = np.array(list(map(int, true_classes)))[select_indexs_test]
    acc_c = round(sum(x == y for x, y in zip(true_classes, predicted_classes)) / len(true_classes),4)
    overall_acc = acc_c + acc_s

    Perfomance = Perfomance.append({'Feature_Extract': "ResNet",'Inputs': "SPEC",'Time':run_time,
                                    'Partcp_Acc':acc_s,'Command_Acc':acc_c}, ignore_index=True)
    
    if high_acc < overall_acc:
        resnet_model.save('Digital_data_Initial_group_model_0915_SPEC.h5')
        high_acc = overall_acc
        best_index = run
        
    del resnet_model
    run = run + 1

Epoch 1/10
16/16 [==============================] - 4s 268ms/step - loss: 3.7532 - participant_output_loss: 1.5756 - command_output_loss: 2.1729 - command_output_1_loss: 1.1308e-04 - participant_output_1_loss: 0.0046 - participant_output_accuracy: 0.3110 - command_output_accuracy: 0.4180 - command_output_1_accuracy: 0.0500 - participant_output_1_accuracy: 0.0640 - val_loss: 3.3657 - val_participant_output_loss: 1.3917 - val_command_output_loss: 1.9737 - val_command_output_1_loss: 3.4384e-05 - val_participant_output_1_loss: 2.5955e-04 - val_participant_output_accuracy: 0.4940 - val_command_output_accuracy: 0.6620 - val_command_output_1_accuracy: 0.0240 - val_participant_output_1_accuracy: 0.0000e+00
Epoch 2/10
16/16 [==============================] - 2s 141ms/step - loss: 3.2116 - participant_output_loss: 1.3142 - command_output_loss: 1.8972 - command_output_1_loss: 3.3628e-05 - participant_output_1_loss: 1.2291e-04 - participant_output_accuracy: 0.5550 - command_output_accuracy: 0.7670

Epoch 3/10
16/16 [==============================] - 2s 120ms/step - loss: 0.6688 - participant_output_loss: 0.1885 - command_output_loss: 0.4803 - command_output_1_loss: 3.7485e-05 - participant_output_1_loss: 1.0945e-05 - participant_output_accuracy: 0.9960 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.6290 - participant_output_1_accuracy: 0.2350 - val_loss: 0.8541 - val_participant_output_loss: 0.3429 - val_command_output_loss: 0.5112 - val_command_output_1_loss: 1.7638e-05 - val_participant_output_1_loss: 1.3298e-05 - val_participant_output_accuracy: 0.9300 - val_command_output_accuracy: 0.9780 - val_command_output_1_accuracy: 0.1120 - val_participant_output_1_accuracy: 0.2180
Epoch 4/10
16/16 [==============================] - 2s 129ms/step - loss: 0.5768 - participant_output_loss: 0.1661 - command_output_loss: 0.4107 - command_output_1_loss: 2.1729e-05 - participant_output_1_loss: 1.1554e-05 - participant_output_accuracy: 0.9960 - command_output_accuracy: 1.0000

Epoch 5/10
16/16 [==============================] - 3s 185ms/step - loss: 2.6221 - participant_output_loss: 0.6519 - command_output_loss: 1.9701 - command_output_1_loss: 2.7866e-05 - participant_output_1_loss: 1.4356e-05 - participant_output_accuracy: 0.9260 - command_output_accuracy: 0.8180 - command_output_1_accuracy: 0.1060 - participant_output_1_accuracy: 0.2820 - val_loss: 2.6138 - val_participant_output_loss: 0.6729 - val_command_output_loss: 1.9408 - val_command_output_1_loss: 2.0083e-05 - val_participant_output_1_loss: 1.3462e-05 - val_participant_output_accuracy: 0.8840 - val_command_output_accuracy: 0.9020 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.3120
Epoch 6/10
16/16 [==============================] - 2s 146ms/step - loss: 2.4501 - participant_output_loss: 0.5460 - command_output_loss: 1.9041 - command_output_1_loss: 5.3612e-05 - participant_output_1_loss: 1.2772e-05 - participant_output_accuracy: 0.9480 - command_output_accuracy: 0.

Epoch 7/10
16/16 [==============================] - 3s 164ms/step - loss: 1.2832 - participant_output_loss: 0.1211 - command_output_loss: 1.1621 - command_output_1_loss: 4.3707e-05 - participant_output_1_loss: 1.0451e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0050 - participant_output_1_accuracy: 0.2410 - val_loss: 1.4202 - val_participant_output_loss: 0.2531 - val_command_output_loss: 1.1670 - val_command_output_1_loss: 3.5984e-05 - val_participant_output_1_loss: 1.2521e-05 - val_participant_output_accuracy: 0.9520 - val_command_output_accuracy: 0.9900 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.1360
Epoch 8/10
16/16 [==============================] - 2s 148ms/step - loss: 1.2070 - participant_output_loss: 0.1091 - command_output_loss: 1.0979 - command_output_1_loss: 5.5773e-05 - participant_output_1_loss: 1.0521e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.

16/16 [==============================] - 3s 160ms/step - loss: 2.3118 - participant_output_loss: 0.5140 - command_output_loss: 1.7977 - command_output_1_loss: 4.0528e-05 - participant_output_1_loss: 1.1065e-05 - participant_output_accuracy: 0.9550 - command_output_accuracy: 0.9780 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.1460 - val_loss: 2.3461 - val_participant_output_loss: 0.5677 - val_command_output_loss: 1.7783 - val_command_output_1_loss: 1.8373e-05 - val_participant_output_1_loss: 1.0454e-05 - val_participant_output_accuracy: 0.9020 - val_command_output_accuracy: 0.9400 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.2000
Epoch 9/10
16/16 [==============================] - 2s 151ms/step - loss: 2.1532 - participant_output_loss: 0.4188 - command_output_loss: 1.7344 - command_output_1_loss: 5.0623e-05 - participant_output_1_loss: 1.0672e-05 - participant_output_accuracy: 0.9800 - command_output_accuracy: 0.9520 - 

...

Epoch 2/10
16/16 [==============================] - 2s 146ms/step - loss: 3.5701 - participant_output_loss: 1.4720 - command_output_loss: 2.0977 - command_output_1_loss: 2.7386e-04 - participant_output_1_loss: 1.1951e-04 - participant_output_accuracy: 0.4780 - command_output_accuracy: 0.4650 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.3510 - val_loss: 3.4822 - val_participant_output_loss: 1.4353 - val_command_output_loss: 2.0467 - val_command_output_1_loss: 1.3992e-04 - val_participant_output_1_loss: 6.2006e-05 - val_participant_output_accuracy: 0.4360 - val_command_output_accuracy: 0.5780 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.4980
Epoch 3/10
16/16 [==============================] - 3s 159ms/step - loss: 3.3712 - participant_output_loss: 1.3588 - command_output_loss: 2.0122 - command_output_1_loss: 1.7897e-04 - participant_output_1_loss: 5.1291e-05 - participant_output_accuracy: 0.5970 - command_output_accuracy

Epoch 4/10
16/16 [==============================] - 2s 131ms/step - loss: 1.4531 - participant_output_loss: 0.3770 - command_output_loss: 1.0760 - command_output_1_loss: 6.4098e-05 - participant_output_1_loss: 2.0346e-05 - participant_output_accuracy: 0.9930 - command_output_accuracy: 0.9990 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.2070 - val_loss: 1.5835 - val_participant_output_loss: 0.5003 - val_command_output_loss: 1.0832 - val_command_output_1_loss: 3.5510e-05 - val_participant_output_1_loss: 2.4408e-05 - val_participant_output_accuracy: 0.9300 - val_command_output_accuracy: 0.9800 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.2220
Epoch 5/10
16/16 [==============================] - 3s 166ms/step - loss: 1.3213 - participant_output_loss: 0.3262 - command_output_loss: 0.9950 - command_output_1_loss: 5.3109e-05 - participant_output_1_loss: 2.0642e-05 - participant_output_accuracy: 0.9930 - command_output_accuracy

Epoch 6/10
16/16 [==============================] - 2s 151ms/step - loss: 2.5714 - participant_output_loss: 0.7157 - command_output_loss: 1.8557 - command_output_1_loss: 4.6101e-05 - participant_output_1_loss: 1.2716e-05 - participant_output_accuracy: 0.9320 - command_output_accuracy: 0.9240 - command_output_1_accuracy: 0.0030 - participant_output_1_accuracy: 0.2090 - val_loss: 2.5823 - val_participant_output_loss: 0.7524 - val_command_output_loss: 1.8299 - val_command_output_1_loss: 2.8268e-05 - val_participant_output_1_loss: 1.3150e-05 - val_participant_output_accuracy: 0.8920 - val_command_output_accuracy: 0.9380 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.1500
Epoch 7/10
16/16 [==============================] - 2s 121ms/step - loss: 2.3990 - participant_output_loss: 0.6139 - command_output_loss: 1.7850 - command_output_1_loss: 7.2045e-05 - participant_output_1_loss: 1.2509e-05 - participant_output_accuracy: 0.9640 - command_output_accuracy: 0.

Epoch 8/10
16/16 [==============================] - 2s 138ms/step - loss: 1.2520 - participant_output_loss: 0.2174 - command_output_loss: 1.0346 - command_output_1_loss: 4.6927e-05 - participant_output_1_loss: 1.1959e-05 - participant_output_accuracy: 0.9970 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.3130 - participant_output_1_accuracy: 0.1750 - val_loss: 1.4130 - val_participant_output_loss: 0.3549 - val_command_output_loss: 1.0580 - val_command_output_1_loss: 3.0982e-05 - val_participant_output_1_loss: 1.4258e-05 - val_participant_output_accuracy: 0.9480 - val_command_output_accuracy: 0.9860 - val_command_output_1_accuracy: 1.0000 - val_participant_output_1_accuracy: 0.1080
Epoch 9/10
16/16 [==============================] - 2s 120ms/step - loss: 1.1738 - participant_output_loss: 0.1985 - command_output_loss: 0.9753 - command_output_1_loss: 3.5627e-05 - participant_output_1_loss: 1.1180e-05 - participant_output_accuracy: 0.9980 - command_output_accuracy: 1.0000

16/16 [==============================] - 3s 162ms/step - loss: 2.0747 - participant_output_loss: 0.5202 - command_output_loss: 1.5544 - command_output_1_loss: 4.4262e-05 - participant_output_1_loss: 1.2552e-05 - participant_output_accuracy: 0.9690 - command_output_accuracy: 0.9660 - command_output_1_accuracy: 0.0070 - participant_output_1_accuracy: 0.2510 - val_loss: 2.1369 - val_participant_output_loss: 0.6087 - val_command_output_loss: 1.5282 - val_command_output_1_loss: 2.8987e-05 - val_participant_output_1_loss: 1.4534e-05 - val_participant_output_accuracy: 0.8940 - val_command_output_accuracy: 0.9400 - val_command_output_1_accuracy: 0.3880 - val_participant_output_1_accuracy: 0.3000
Epoch 10/10
16/16 [==============================] - 3s 158ms/step - loss: 1.9185 - participant_output_loss: 0.4541 - command_output_loss: 1.4643 - command_output_1_loss: 3.9442e-05 - participant_output_1_loss: 1.2754e-05 - participant_output_accuracy: 0.9770 - command_output_accuracy: 0.9890 - command

In [15]:
Perfomance

,Command_Acc,Feature_Extract,Inputs,Partcp_Acc,Time
0,0.982,ResNet,SPEC,0.970,53.745848
1,0.980,ResNet,SPEC,0.968,59.788192
2,0.972,ResNet,SPEC,0.960,60.039808
3,0.982,ResNet,SPEC,0.958,56.502710
4,0.980,ResNet,SPEC,0.968,56.194534
5,0.986,ResNet,SPEC,0.966,55.374021
6,0.978,ResNet,SPEC,0.952,55.079770
7,0.988,ResNet,SPEC,0.960,55.301056
8,0.984,ResNet,SPEC,0.960,51.448608
9,0.980,ResNet,SPEC,0.952,52.307214


In [16]:
Perfomance.to_csv('Performance_0915_Digit_Inputs.csv',index=False)